#### Importing Libraries

In [1]:
import numpy as np 
import pandas as pd

import html5lib
import lxml

print("First set libraries imported. numpy, pandas, html5lib, lxml")

pd.set_option('display.max_columns', 999)

First set libraries imported. numpy, pandas, html5lib, lxml


#### Installing geopy module

In [2]:
#
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab


Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2019.6.16  |       hecc5488_0         145 KB  conda-forge
    certifi-2018.8.24          |        py35_1001         139 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    openssl-1.0.2r             |       h14c3975_0         3.1 MB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         3.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

   

####### importing matpotlib, Nominatim and Kmeans library
####### and installing kmeans library

In [3]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.pyplot as plt
# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

###################ALL Libraries#########################
print ("Hello Capstone Project Course! Libraries import completed!!!")

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/DSX-Python35

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    altair-2.2.2               |           py35_1         462 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         560 KB

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge


vincent-0.4.4        | 28 KB     | #############################

In [4]:
page = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

### The dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
df_pincodes0 = pd.read_html(page, attrs={"class":"wikitable"})[0]

#print("Step001")

#df_pincodes.columns = df_pincodes.iloc[0]
#df_pincodes.drop(df_pincodes.index[0])

headers = df_pincodes0.iloc[0]
df_pincodes  = pd.DataFrame(df_pincodes0.values[1:], columns=headers)

print(df_pincodes)




0   Postcode           Borough  \
0        M1A      Not assigned   
1        M2A      Not assigned   
2        M3A        North York   
3        M4A        North York   
4        M5A  Downtown Toronto   
5        M5A  Downtown Toronto   
6        M6A        North York   
7        M6A        North York   
8        M7A      Queen's Park   
9        M8A      Not assigned   
10       M9A         Etobicoke   
11       M1B       Scarborough   
12       M1B       Scarborough   
13       M2B      Not assigned   
14       M3B        North York   
15       M4B         East York   
16       M4B         East York   
17       M5B  Downtown Toronto   
18       M5B  Downtown Toronto   
19       M6B        North York   
20       M7B      Not assigned   
21       M8B      Not assigned   
22       M9B         Etobicoke   
23       M9B         Etobicoke   
24       M9B         Etobicoke   
25       M9B         Etobicoke   
26       M9B         Etobicoke   
27       M1C       Scarborough   
28       M1C  

In [5]:
### Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.
new_df = df_pincodes[df_pincodes.Borough != 'Not assigned'] 
print (new_df)
print("Step002")

### new_df = new_df[new_df.Neighbourhood != 'Not assigned'] ### No need to filter this
new_df = new_df.reset_index().drop(['index'], axis=1)
print("Step003")

### More than one neighborhood can exist in one postal code area. 
### For example, in the table on the Wikipedia page, you will notice 
### that M5A is listed twice and has two neighborhoods: Harbourfront 
### and Regent Park. These two rows will be combined into one row with 
### the neighborhoods separated with a comma as shown in row 11 in the above table.

df_grpby = new_df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(lambda Neighbourhoods: ','.join(Neighbourhoods)).reset_index()
#df_grpby = new_df.groupby([0, 1])[2].apply(lambda Neighbourhoods: ','.join(Neighbourhoods)).reset_index()
print("Step004")
print(df_grpby)



0   Postcode           Borough  \
2        M3A        North York   
3        M4A        North York   
4        M5A  Downtown Toronto   
5        M5A  Downtown Toronto   
6        M6A        North York   
7        M6A        North York   
8        M7A      Queen's Park   
10       M9A         Etobicoke   
11       M1B       Scarborough   
12       M1B       Scarborough   
14       M3B        North York   
15       M4B         East York   
16       M4B         East York   
17       M5B  Downtown Toronto   
18       M5B  Downtown Toronto   
19       M6B        North York   
22       M9B         Etobicoke   
23       M9B         Etobicoke   
24       M9B         Etobicoke   
25       M9B         Etobicoke   
26       M9B         Etobicoke   
27       M1C       Scarborough   
28       M1C       Scarborough   
29       M1C       Scarborough   
31       M3C        North York   
32       M3C        North York   
33       M4C         East York   
34       M5C  Downtown Toronto   
35       M6C  

In [6]:
c = new_df.loc[new_df['Neighbourhood'] == 'Not assigned'].index.values[0]
print(c)
print(new_df.loc[c])
print(new_df.loc[c].Borough)

new_df.at[c, 'Neighbourhood'] = new_df.loc[c].Borough
print(new_df)

new_df.shape

6
0
Postcode                  M7A
Borough          Queen's Park
Neighbourhood    Not assigned
Name: 6, dtype: object
Queen's Park
0   Postcode           Borough  \
0        M3A        North York   
1        M4A        North York   
2        M5A  Downtown Toronto   
3        M5A  Downtown Toronto   
4        M6A        North York   
5        M6A        North York   
6        M7A      Queen's Park   
7        M9A         Etobicoke   
8        M1B       Scarborough   
9        M1B       Scarborough   
10       M3B        North York   
11       M4B         East York   
12       M4B         East York   
13       M5B  Downtown Toronto   
14       M5B  Downtown Toronto   
15       M6B        North York   
16       M9B         Etobicoke   
17       M9B         Etobicoke   
18       M9B         Etobicoke   
19       M9B         Etobicoke   
20       M9B         Etobicoke   
21       M1C       Scarborough   
22       M1C       Scarborough   
23       M1C       Scarborough   
24       M3C        

(211, 3)

In [7]:
##### Complete for top 3 questions.

##### tried with four square api a lot. It is timing out most of the times. so downloaded the file below to move forward...

In [8]:
##### Download csvfile to pandas dataframe.

df_csv = pd.read_csv("https://cocl.us/Geospatial_data")

df_csv.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [9]:
##### merge data frames

print(df_csv.columns)
print(new_df.columns)

new_df.rename(columns={"Postcode":"Postal Code"}, inplace=True)
print(df_csv.columns)
print(new_df.columns)
new_df.head()

df_new_merged = pd.merge(new_df, df_csv, on='Postal Code', how='left')

print("displaying merged dataset:")
print(df_new_merged)

Index(['Postal Code', 'Latitude', 'Longitude'], dtype='object')
Index(['Postcode', 'Borough', 'Neighbourhood'], dtype='object', name=0)
Index(['Postal Code', 'Latitude', 'Longitude'], dtype='object')
Index(['Postal Code', 'Borough', 'Neighbourhood'], dtype='object', name=0)
displaying merged dataset:
    Postal Code           Borough  \
0           M3A        North York   
1           M4A        North York   
2           M5A  Downtown Toronto   
3           M5A  Downtown Toronto   
4           M6A        North York   
5           M6A        North York   
6           M7A      Queen's Park   
7           M9A         Etobicoke   
8           M1B       Scarborough   
9           M1B       Scarborough   
10          M3B        North York   
11          M4B         East York   
12          M4B         East York   
13          M5B  Downtown Toronto   
14          M5B  Downtown Toronto   
15          M6B        North York   
16          M9B         Etobicoke   
17          M9B         Etobicok

In [10]:
##### showing this again with top head rows only
df_new_merged.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M5A,Downtown Toronto,Regent Park,43.654260,-79.360636
4,M6A,North York,Lawrence Heights,43.718518,-79.464763
